Coming up with an algorithm to create target rhythm from original rhythm such that real-time time-stretching (or even real-time stuttering) is possible

In [17]:
import librosa
import numpy as np

import pardir; pardir.pardir() # Allow imports from parent directory
import bjorklund
import fibonaccistretch

In [2]:
librosa.effects.time_stretch??
librosa.core.phase_vocoder??
fibonaccistretch.euclidean_stretch??

In [8]:
# Generate rhythms based on parameters
def generate_original_and_target_rhythms(num_pulses, original_length, target_length):
    original_rhythm = bjorklund.bjorklund(pulses=num_pulses, steps=original_length)
    target_rhythm = bjorklund.bjorklund(pulses=len(original_rhythm), steps=target_length)
    return (original_rhythm, target_rhythm)

original_rhythm, target_rhythm = generate_original_and_target_rhythms(3, 8, 13)

"Original rhythm: {}    Target rhythm: {}".format(original_rhythm, target_rhythm)

'Original rhythm: [1, 0, 0, 1, 0, 0, 1, 0]    Target rhythm: [1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1]'

In [11]:
lcm = (8*13) / fibonaccistretch.euclid(8, 13)
lcm

104

In [12]:
8*13

104

Use LCM to "equalize" rhythms so that they're of equal length.

e.g.

`a = [1,0,0,1]`



`b = [1,1,0]`

become

`equalized_a = [1,-,-,0,-,-,0,-,-,1,-,-]`



`equalized_b = [1,-,-,-,1,-,-,-,0,-,-,-]`

In [29]:
# "Equalize" (i.e. scale rhythms so they're of equal length)
def equalize_rhythm_subdivisions(original_rhythm, target_rhythm, delimiter="-"):
    original_length = len(original_rhythm)
    target_length = len(target_rhythm)
    lcm = (original_length*target_length) / fibonaccistretch.euclid(original_length, target_length)
    original_scale_factor = (lcm / original_length) - 1
    target_scale_factor = (lcm / target_length) - 1
    
    print("lcm={}, original_scale_factor={}, target_scale_factor={}").format(lcm, original_scale_factor, target_scale_factor)
    
    delimiter = str(delimiter)
    original_rhythm = list((delimiter*original_scale_factor).join([str(x) for x in original_rhythm]))
    target_rhythm = list((delimiter*target_scale_factor).join([str(x) for x in target_rhythm]))
    
    original_rhythm.extend(list(delimiter*original_scale_factor))
    target_rhythm.extend(list(delimiter*target_scale_factor))

    
    return (original_rhythm, target_rhythm)

# print(scale_rhythm_subdivisions(original_rhythm, target_rhythm))
original_rhythm, target_rhythm = generate_original_and_target_rhythms(3, 8, 13)
print("Original rhythm: {}    Target rhythm: {}".format(original_rhythm, target_rhythm))
equalized_original_rhythm, equalized_target_rhythm = equalize_rhythm_subdivisions(original_rhythm, target_rhythm)
(len(equalized_original_rhythm), len(equalized_target_rhythm))

Original rhythm: [1, 0, 0, 1, 0, 0, 1, 0]    Target rhythm: [1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1]
lcm=104, original_scale_factor=12, target_scale_factor=7


(104, 104)

Get pulse indices so we can see how the equalized original and target relate. In particular, our goal is to create a relationship such that the original pulse indices always come first (so that they're bufferable in real-time)

In [26]:
def get_pulse_indices_for_rhythm(rhythm, pulse_symbols=[1]):
    pulse_symbols = [str(s) for s in pulse_symbols]
    rhythm = [str(x) for x in rhythm]
    pulse_indices = [i for i,symbol in enumerate(rhythm) if symbol in pulse_symbols]
    return pulse_indices

equalized_original_pulse_indices = get_pulse_indices_for_rhythm(equalized_original_rhythm)
equalized_target_pulse_indices = get_pulse_indices_for_rhythm(equalized_target_rhythm)
(equalized_original_pulse_indices, equalized_target_pulse_indices)

([0, 39, 78], [0, 16, 24, 40, 56, 64, 80, 96])

For original we'll actually use ALL the steps instead of just pulses though. So:

In [28]:
equalized_original_pulse_indices = get_pulse_indices_for_rhythm(equalized_original_rhythm, [1,0])
equalized_target_pulse_indices = get_pulse_indices_for_rhythm(equalized_target_rhythm, [1])
print(equalized_original_pulse_indicesequalized_target_pulse_indices)

[0, 13, 26, 39, 52, 65, 78, 91]
[0, 16, 24, 40, 56, 64, 80, 96]


Now we can check to see if all the original pulse indices come first (this is our goal):

In [35]:
for i in range(len(equalized_original_pulse_indices)):
    opi = equalized_original_pulse_indices[i]
    tpi = equalized_target_pulse_indices[i]
    if (opi > tpi):
        print("Oh no; original pulse at {} comes after target pulse at {} (diff={})".format(opi, tpi, opi-tpi))

Oh no; original pulse at 26 comes after target pulse at 24 (diff=2)
Oh no; original pulse at 65 comes after target pulse at 64 (diff=1)


Oh no...

In [18]:
np.indices?

In [14]:
"0"*4
"-".join([str(x) for x in [2,3,4]])

'2-3-4'

In [15]:
print("a"); print("b")

a
b
